In [1]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import applications


In [2]:
vgg=applications.vgg16.VGG16()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
cnn=Sequential()
for capa in vgg.layers:
    cnn.add(capa)

In [5]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
cnn.pop()

In [7]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [8]:
for layer in cnn.layers:
    layer.trainable=False

In [9]:
cnn.add(Dense(2,activation='softmax'))

In [10]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [11]:
def modelo():
    vgg=applications.vgg16.VGG16()
    cnn=Sequential()
    for capa in vgg.layers:
        cnn.add(capa)
    cnn.layers.pop()
    for layer in cnn.layers:
        layer.trainable=False
    cnn.add(Dense(2,activation='softmax'))
    
    return cnn

In [12]:
# Import required libraries for this section

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import math
import cv2                     # OpenCV library for computer vision
from PIL import Image
import time

from keras.models import load_model


# Auxiliary functions

def read_image(path):
    """ Method to read an image from file to matrix """
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def plot_image(image, title=''):
    """ It plots an image as it is in a single column plot """
    # Plot our image using subplots to specify a size and title
    fig = plt.figure(figsize = (8,8))
    ax1 = fig.add_subplot(111)
    ax1.set_xticks([])
    ax1.set_yticks([])

    ax1.set_title(title)
    ax1.imshow(image)



Using TensorFlow backend.


In [13]:
def get_faces(image):
    """
    It returns an array with the detected faces in an image
    Every face is defined as OpenCV does: top-left x, top-left y, width and height.
    """
    # To avoid overwriting
    image_copy = np.copy(image)
    
    # The filter works with grayscale images
    gray = cv2.cvtColor(image_copy, cv2.COLOR_RGB2GRAY)

    # Extract the pre-trained face detector from an xml file
    face_classifier = cv2.CascadeClassifier('detectors/haarcascade_frontalface_default.xml')
    
    # Detect the faces in image
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)
    
    return faces 

def draw_faces(image, faces=None, plot=True):
    """
    It plots an image with its detected faces. If faces is None, it calculates the faces too
    """
    if faces is None:
        faces = get_faces(image)
    
    # To avoid overwriting
    image_with_faces = np.copy(image)
    
    # Get the bounding box for each detected face
    for (x,y,w,h) in faces:
        # Add a red bounding box to the detections image
        cv2.rectangle(image_with_faces, (x,y), (x+w,y+h), (255,0,0), 3)
        
    if plot is True:
        plot_image(image_with_faces)
    else:
        return image_with_faces
    
    

def plot_image_with_keypoints(image, image_info):
    """
    It plots keypoints given in (x,y) format
    """
    fig = plt.figure(figsize = (8,8))
    ax1 = fig.add_subplot(111)
    
    for (face, keypoints) in image_info:
        for (x,y) in keypoints:
            ax1.scatter(x, y, marker='o', c='c', s=10)
   

    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.imshow(image)

In [14]:
def get_keypoints(image, faces=None):
    
    # list of pairs (face, keypoints)
    result = []
    
    if faces is None:
        faces = get_faces(image)
    
    # Same size than training/validation set
    faces_shape = (96, 96)
    
    # To avoid overwriting
    image_copy = np.copy(image)
    
    # For each face, we detect keypoints and show features
    for (x,y,w,h) in faces:

        # We crop the face region
        face = image_copy[y:y+h,x:x+w]

        # Face converted to grayscale and resize (our CNN receives images of 96x96x1)
        gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        resize_gray_face = cv2.resize(gray_face, faces_shape) / 255

        # Formatting x inputs. Inputs will have format of (1, 96, 96, 1)
        inputs = np.expand_dims(np.expand_dims(resize_gray_face, axis=-1), axis=0)
                                
        # Get keypoints result                        
        predicted_keypoints = model.predict(inputs)

        # All keypoints in a single flat array. We will retrieve keypoints as (x,y) with (idx, idx+1) values.
        predicted_keypoints = np.squeeze(predicted_keypoints)
        
        keypoints = []        
        for idx in range(0, len(predicted_keypoints), 2):
            # Scale factor (revert scale)
            x_scale_factor = face.shape[0]/faces_shape[0] 
            y_scale_factor = face.shape[1]/faces_shape[1] 

            # Offset of the center of the scatter
            x_center_left_offset = predicted_keypoints[idx] * faces_shape[0]/2 + faces_shape[0]/2 
            y_center_left_offset = predicted_keypoints[idx + 1] * faces_shape[1]/2 + faces_shape[1]/2
            
            x_center = int(x + (x_scale_factor * x_center_left_offset))
            y_center = int(y + (y_scale_factor * y_center_left_offset))

            keypoints.append([x_center, y_center])
        
        result.append([(x,y,w,h), keypoints])
    
    return result

def show_image_and_features(image_path):
    image = read_image(image_path)
    faces = get_faces(image)
    keypoints = get_keypoints(image, faces)
    image_with_faces = draw_faces(image, faces ,plot=False)
    plot_image_with_keypoints(image_with_faces, keypoints)



In [15]:
#Ahora el codigo de entrenamiento 
K.clear_session()

import os
import numpy as np
import cv2 

def read_image(path):
    """ Method to read an image from file to matrix """
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def get_faces(image):
    """
    It returns an array with the detected faces in an image
    Every face is defined as OpenCV does: top-left x, top-left y, width and height.
    """
    # To avoid overwriting
    image_copy = np.copy(image)
    
    # The filter works with grayscale images
    gray = cv2.cvtColor(image_copy, cv2.COLOR_RGB2GRAY)

    # Extract the pre-trained face detector from an xml file
    face_classifier = cv2.CascadeClassifier('detectors/haarcascade_frontalface_default.xml')
    
    # Detect the faces in image
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)
    
    return faces 

entrenamiento = "/home/sebastian/Documentos/Tesis3/Tesis/CodigoNeuronaRF/ai-reconocimiento-facial-python-master/tesis/entrenamiento"
#directorio ="/home/sebastian/Documentos/Tesis3/Tesis/CodigoOrdenado/datos-101/entrenamiento1"

'''if not os.path.exists(directorio):
    os.mkdir(directorio)
lista_de_carpetas = os.listdir(entrenamiento)    
for car in lista_de_carpetas:
    dire = entrenamiento+'/'+car
    lista_de_archivos = os.listdir(dire)
    for ar in lista_de_archivos:
        image = read_image(entrenamiento+'/'+car+'/'+ar)
        faces = get_faces(image)
        for (x,y,w,h) in faces:
            roi_color = image[y:y+h, x:x+w]
            if not os.path.exists(directorio):
                os.mkdir(directorio)
            if not os.path.exists(directorio+'/'+car):
                os.mkdir(directorio+'/'+car)
                print('listoco' + directorio+'/'+car)
            img_item = str(directorio+'/'+car+'/'+ar)
            cv2.imwrite(img_item, roi_color)
'''
data_entrenamiento = entrenamiento
#data_validacion = './data-profes/validacion'




#El mas Optimo hasta el momento
epocas= 10
longitud, altura = 224, 224
batch_size = 15
pasos = 24
validation_steps = 15
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 2
#lr = 0.0004
lr = 0.003

##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip= True,
    vertical_flip = True,
    validation_split=0.2,
    rotation_range= 45)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

#validacion_generador = test_datagen.flow_from_directory(
#    data_validacion,
#    target_size=(altura, longitud),
#    batch_size=batch_size,
#    class_mode='categorical')


##CREAR LA RED VGG16

cnn=modelo()

cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])


cnn.fit_generator(
    entrenamiento_generador, 
    steps_per_epoch = pasos, 
    epochs = epocas)
    #validation_data = validacion_generador, 
    #validation_steps = validation_steps)


target_dir = './TesisModel/'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)
cnn.save('./TesisModel/modelo.h5')
cnn.save_weights('./TesisModel/pesos.h5')
print("Finishim!!")

Found 12 images belonging to 2 classes.
Epoch 1/10
24/24 [==============================] - 116s 5s/step - loss: 0.6906 - acc: 0.7778
Epoch 2/10
24/24 [==============================] - 127s 5s/step - loss: 0.6845 - acc: 0.9653
Epoch 3/10
24/24 [==============================] - 121s 5s/step - loss: 0.6781 - acc: 0.9444
Epoch 4/10
24/24 [==============================] - 119s 5s/step - loss: 0.6723 - acc: 0.9410
Epoch 5/10
24/24 [==============================] - 120s 5s/step - loss: 0.6655 - acc: 0.9479
Epoch 6/10
24/24 [==============================] - 120s 5s/step - loss: 0.6579 - acc: 0.9583
Epoch 7/10
24/24 [==============================] - 119s 5s/step - loss: 0.6501 - acc: 0.9549
Epoch 8/10
24/24 [==============================] - 120s 5s/step - loss: 0.6473 - acc: 0.9132
Epoch 9/10
24/24 [==============================] - 119s 5s/step - loss: 0.6356 - acc: 0.9618
Epoch 10/10
24/24 [==============================] - 121s 5s/step - loss: 0.6358 - acc: 0.9375
Finishim!!
